# Data Preparation

In [15]:
import unidecode
import string
import random
import re
import json

all_characters = string.printable
n_characters = len(all_characters)
chunk_len = 20

In [16]:
job_texts = json.load(open('./data/final/jobs.json', 'r'))

To make inputs out of this big string of data, we will be splitting it into chunks.

In [17]:
def random_chunk():
    file = random.choice(job_texts)
    start = 0
    end = len(file)-1
    return file[start:end]

print(random_chunk())

Hi there! Do you want a good future, grow in money earning power, and the respect of all? Today only: We can assist with Diplomas from prestigious universities based on your present knowledge and work experience. Get a Degree in 5 weeks with our program! ~Our program will let ANYONE with professional experience get a 100% verified Degree: ~Doctorate ~Bachelors ~Masters - Think about it... - Follow YOUR Dreams! - Live a wonderful life by earning or upgrading your degree. This is a nice way to make a right move and receive your due benefits... if you are qualified but are lacking that piece of paper. Get one from us in a short time. If you want to get better - you must Contact Us 24/7 to start improving your life! ~CONTACT US FOR A FREE CONSULTATION~ 1-301-396-3506 Please leave us a voice message with your name and phone number with country code if outside USA and we'll get back to you as soon as possible. It is your decision... Make the right move. Best wishes. Do Not Reply to this Emai

In [18]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    print(inp, target)
    return inp, target

In [19]:
random_chunk()

'Hey bud!! Do you want an effective future, soar in money earning power, and pat on the back :)? Today only: We can assist with Diplomas from prestigious universities based on your present knowledge and work experience. Get a Degree in 5 weeks with our program! ~Our program will let EVERYONE with professional experience gain a 100% verified Degree: ~Doctorate ~Bachelors ~Masters - Just think about it... - Realize YOUR Dreams! - Live a better life by earning or upgrading your degree. This is a nice way to make a right move and receive your due benefits... if you are qualified but are lacking that piece of paper. Get one from us in a short time. Contact Us to start improving your life! ~CALL FOR A FREE CONSULTATION~ 1-301-396-3506 You must leave us a voice message with your name and phone number with country code if outside USA and we will call you as soon as possible. It is your decision... Make the right decision. Sincerely. Do Not Reply to this Email. We do not reply to text inquiries

# Model building

In [20]:
import torch
import torch.nn as nn
from torch.autograd import Variable


class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1).cuda())
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size)).cuda()

# Inputs and Targets

In [21]:
# Turn string into list of longs
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)

print(char_tensor('abcDEF'))

tensor([10, 11, 12, 39, 40, 41])


In [22]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

# Evaluation


In [23]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden()
    prime_input = char_tensor(prime_str)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)

    return predicted

# Training

In [24]:
import time, math

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [25]:
def train(inp, target):
    hidden = decoder.init_hidden()
    decoder.zero_grad()
    loss = torch.tensor(0).cuda()

    for c in range(len(inp)):
        output, hidden = decoder(inp[c].cuda(), hidden)
        tr = torch.Tensor([target[c]]).long().cuda()
        output = output.float()
        loss = loss + criterion(output, tr)
    if loss > 0:
        loss.backward()
    decoder_optimizer.step()

    return loss.item() / chunk_len

In [37]:
capital_letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [41]:
from tqdm import tqdm

n_epochs = 2000
print_every = 10
plot_every = 10
hidden_size = 100
n_layers = 3
lr = 0.005

decoder = RNN(n_characters, hidden_size, n_characters, n_layers).cuda()
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in tqdm(range(1, n_epochs + 1)):
    loss = train(*random_training_set())  
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs , loss))
        print(evaluate('H', 100), '\n')

    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

  0%|          | 10/2000 [00:17<1:12:02,  2.17s/it]

 17s (10 0%) 192.6812]
H#nn ap iol ta  h m    mpcte. eaw geuna .nuarml gw r r y eoe hfraw upors.trama Nh Rna tl  serfe r  a. 



  1%|          | 20/2000 [00:36<1:13:20,  2.22s/it]

 36s (20 0%) 86.0210]
H7feawnot oe neo   set deoiy ey mtzki1lsfr i er adlte  n r  ono A ponaooniltiiwss  o:l  deeeasna~dsu  



  2%|▏         | 30/2000 [00:53<1:02:32,  1.90s/it]

...

 23s (1980 0%) 9.4772]
Helle! Do you want a suital all you back to you bushht of pat on the to all but it with Diplomas from 



100%|█████████▉| 1990/2000 [45:33<00:09,  1.00it/s]

 33s (1990 0%) 22.9609]
Hi proyour name and the recognition, of at to work experience. Get a Degree in 4 weeks with our progr 



100%|██████████| 2000/2000 [45:47<00:00,  1.37s/it]

 47s (2000 1%) 9.0094]
Hey bud!! Do you want a superior future, soar in money earning ore apprente but in in money earning p 



# Training Loss

In [1]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

plt.figure()
plt.plot(all_losses)

NameError: name 'all_losses' is not defined

<Figure size 432x288 with 0 Axes>

# Generating spam at different "temperatures"

In [219]:
print(evaluate('Th', 200, temperature=0.8))

The Uante with and. Bay ince and Outy me ba ank mienckepeid io werial: exeo ot expelii exIan Dill me to flt are the indt heutifp illl ran Uegive a bith fegsie forfiel cet-d incasy incincempe a deddirsit


In [220]:
print(evaluate('Th', 200, temperature=0.2))

The Uave to contrle to unk a with a the USA: +100-00003-509-200001 - 8044 o diploma and an experint the periflt you day we a the the belp you and to and and and and and and On expe weleeit he and with t


In [1]:
print(evaluate('Th', 200, temperature=1.4))

NameError: name 'evaluate' is not defined